In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.autograd import Variable
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error
from torch.utils.data import DataLoader, TensorDataset
import string
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import sklearn.metrics as metrics
from IPython.display import clear_output
import re as re

device = torch.device('cuda')
RANDOM_SEED = 0
EMBEDDING_DIM = 100
HIDDEN_DIM = 100
NUM_HIDDEN_LAYERS = 2
LEARNING_RATE = 0.001
NUM_EPOCH = 100
torch.manual_seed(RANDOM_SEED)

def apply_reg_ex(text):
    matches = re.findall('[a-zа-яА-ЯA-Z0-9\s]+', text)
    res_str = ''
    for each_str in matches:
        res_str = res_str + each_str
    return res_str

def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))
    

def get_tokenized_tensor(income_x, income_y,char_dict, MAX_SEN_LENGHT,DF_LENGHT,class_dict):
    res_tensor_train = torch.zeros([DF_LENGHT,MAX_SEN_LENGHT],dtype=int)
    if income_y is not None:
        res_tensor_labels = torch.zeros([DF_LENGHT],dtype=torch.long)
    else: res_tensor_labels = None
    for df_index in range(DF_LENGHT):
        for index, each_char in enumerate(income_x.iloc[df_index].name_before):
            res_tensor_train[df_index,index] = char_dict[each_char]
        if income_y is not None:
            res_tensor_labels[df_index] = income_y.iloc[df_index]-1
    return res_tensor_train, res_tensor_labels  

all_possible_chars = string.printable + 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + '№–…’«»—ø°±²∆і―'

df =pd.read_excel('/kaggle/input/bit-nom/norm.xlsx')
df = df.rename(columns={"Наименование исходное": "name_before", 
                        "Наименование нормализованное": 
                        "name_after", 
                        'Код':'code',
                        'Класс':'class_nom'})
df = df.sample(frac=1) 
df['name_before'] = df['name_before'].apply(apply_reg_ex)

unique_classes = df.class_nom.unique()
class_dict = {}
number = 1
for each_element in unique_classes:
    class_dict[each_element] = number
    number=number + 1

chars_dict = {'padding':0}
num_char = 1
for each_char in all_possible_chars:
    chars_dict[each_char] = num_char
    num_char+=1


df.class_nom = df.class_nom.map(class_dict)
df.name_after = df.name_after.str.lower()
df.name_before = df.name_before.str.lower()

aggregated_df = df.groupby(['class_nom'],as_index=False).count()
df_x = df.drop(['class_nom'],axis=1)
df_y = df['class_nom'] 
train_x,test_x, train_y,test_y = train_test_split(df_x, df_y,test_size=0.2, random_state=RANDOM_SEED)   

MAX_SEN_LENGHT = 246#max(df.name_before.str.len())
DF_TRAIN_LENGHT = len(train_x)
DF_TEST_LENGHT = len(test_x)
tensor_x, tensor_y = get_tokenized_tensor(train_x, train_y ,chars_dict,MAX_SEN_LENGHT,DF_TRAIN_LENGHT,class_dict)
train_dataset = TensorDataset(tensor_x, tensor_y)
train_dataloader = DataLoader(train_dataset,batch_size=300,drop_last=True)          

val_x, val_y = get_tokenized_tensor(test_x, test_y ,chars_dict,MAX_SEN_LENGHT,DF_TEST_LENGHT,class_dict)


    


In [ ]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x
    
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size , max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        #logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        #logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits_flat
    
def get_key(val):
   
    for key, value in class_dict.items():
        if (val+1) == value:
            return key
 
    return "key doesn't exist"
      

In [ ]:
model = SingleTokenPOSTagger(len(chars_dict), len(class_dict), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.CrossEntropyLoss()

In [ ]:
def evaluate_metrics(model,
                    tensor_x,
                    tensor_y,
                    macro_precision_history,
                    macro_recall_history,
                    macro_f1score_history):
    model.eval()
    train_pred = model(tensor_x)  
    train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(tensor_y))
    #print('Среднее значение функции потерь на обучении', float(train_loss))
    results = metrics.classification_report(tensor_y.view(-1).cpu(), train_pred.argmax(1).reshape(-1).cpu(), \
                                            output_dict=True,\
                                            target_names=class_dict,)
    
    print(metrics.classification_report(tensor_y.view(-1).cpu(), train_pred.argmax(1).reshape(-1).cpu(),
                                            target_names=class_dict))
    

    
    macro_precision_history.append(results['macro avg']['precision'])
    macro_recall_history.append(results['macro avg']['recall'])
    macro_f1score_history.append(results['macro avg']['f1-score'])
    
    

In [ ]:
macro_precision_history_train = []
macro_recall_history_train = []
macro_f1score_history_train = []
macro_precision_history_val = []
macro_recall_history_val = []
macro_f1score_history_val = []
epochs_history = []

for i in range(NUM_EPOCH):
    model.train()
    model.to(device)

    for index, (inputs, labels) in enumerate(train_dataloader):
        
        inputs = copy_data_to_device(inputs, device)
        labels = copy_data_to_device(labels, device)

        #inputs = inputs.permute(1,0)
        preds = model(inputs) 
        #_, predicted_labels = torch.max(preds, 1)
        #loss_value = loss_function(preds, labels)
        loss = loss_function(preds, labels)
        optimizer.zero_grad()
        #prec_score = metrics.precision_score(preds.argmax(dim=1).detach().numpy(), \
        #                                      labels.argmax(dim=1).detach().numpy(),average='macro')
        
        #rec_score = metrics.recall_score(preds.argmax(dim=1).detach().numpy(), \
        #                                      labels.argmax(dim=1).detach().numpy(),average='macro')
        #print('Epoch ' + str(i) + ' index ' + str(index) + ' ' + str(loss.item()))
        #print(prec_score)
        #print(rec_score)
        loss.backward()
        optimizer.step()
    
    tensor_x = copy_data_to_device(tensor_x, device)
    tensor_y = copy_data_to_device(tensor_y, device)
    val_x = copy_data_to_device(val_x, device)
    val_y = copy_data_to_device(val_y, device)
    
    clear_output()
    evaluate_metrics(model, 
                     tensor_x,
                     tensor_y,
                     macro_precision_history_train,
                     macro_recall_history_train,
                     macro_f1score_history_train)
    
    evaluate_metrics(model, 
                     val_x,
                     val_y,
                     macro_precision_history_val,
                     macro_recall_history_val,
                     macro_f1score_history_val)

    epochs_history.append(i)
  
    plt.plot(epochs_history, macro_precision_history_train,label='Train')
    plt.plot(epochs_history, macro_precision_history_val,label='Val')
    plt.xlabel('epochs')
    plt.ylabel('macro_precision')
    plt.show()

    plt.plot(epochs_history, macro_recall_history_train,label='Train')
    plt.plot(epochs_history, macro_recall_history_val,label = 'Val')
    plt.xlabel('epochs')
    plt.ylabel('macro_recall')
    plt.show()

    plt.plot(epochs_history, macro_f1score_history_train,label = 'Train')
    plt.plot(epochs_history, macro_f1score_history_val,label = 'Val')
    plt.xlabel('epochs')
    plt.ylabel('macro_f1_score')
    plt.show()
 
    print()
   

In [ ]:
test_df = pd.read_excel('/kaggle/input/bit-nom/norm.xlsx',sheet_name = 'Грязные данные для классификаци')
test_df = test_df.rename(columns={"Наименование исходное": "name_before"})
test_df.name_before = test_df.name_before.str.lower()
test_df['name_before'] = test_df['name_before'].apply(apply_reg_ex)
test_df = test_df.sample(frac=1) 
test_tensor, _ = get_tokenized_tensor(test_df, None ,chars_dict,MAX_SEN_LENGHT,len(test_df),class_dict)
model.eval()
test_tensor = copy_data_to_device(test_tensor, device)
test_preds = model(test_tensor)
test_preds = test_preds.argmax(dim=1).cpu().detach().numpy() 
res_pd = pd.DataFrame(test_preds)
res_pd = res_pd.rename(columns={0: "class_nom"})

In [ ]:
test_df

In [ ]:
res_pd = res_pd.class_nom.apply(get_key)

In [ ]:
test_df.to_csv("output1.xlsx")  
res_pd.to_csv("output2.xlsx")  